# Code for calculating the final dataframes for regression of the effect of decisions on the flow of momentum.

In [ ]:
#five-minute time windows for pitch control and PCV
import os
from progressbar import ProgressBar
import pandas as pd
import pickle

#create a list with all pitch control matches
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code'
pc_matches = []
for file in os.listdir(path):
    if 'pc_over_last_five_minutes' in file:
        pc_matches.append(file[25:32])

#create empty dataframe
final_df = pd.DataFrame()
pbar = ProgressBar()
#iterate over each match in the list
for game in pbar(pc_matches):
    
    #import the pitch control and PCV file
    infile = open('pc_over_last_five_minutes' + game + '.p','rb')
    df_pc = pickle.load(infile)
    infile.close()
    
    #create first and second half dataframes
    fh = df_pc.loc[df_pc['Half'] == 1]
    sh = df_pc.loc[df_pc['Half'] == 2]
    
    #import the decision making dataframe
    infile = open('Decision matches dataframe.p','rb')
    df_decisions = pickle.load(infile)
    infile.close()
    
    #select only the decisions made in the second half for each match
    df_decisions = df_decisions.loc[df_decisions['Match'] == game]
    df_decisions = df_decisions.loc[df_decisions['half'] == '2']
    df_decisions = df_decisions[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'formation_change', 'home score', 'away score', 'current_formation_home', 'current_formation_away', 'players involved', 'Type of decision', 'Match']]
    df_decisions['minute'] = pd.to_numeric(df_decisions['minute'])
    df_decisions = df_decisions.loc[df_decisions['minute'] >= 50]

    
    mean_pc_b_home = []
    mean_wpc_b_home = []
    mean_pc_a_home = []
    mean_wpc_a_home = []
    mean_pc_b_away = []
    mean_wpc_b_away = []
    mean_pc_a_away = []
    mean_wpc_a_away = []
    index_min_below = []
    index_min_above = []
    
    #for each row in the decision dataframe: add pitch control and PCV in the timewindows for the home and way team to the dataframe
    for i in range(len(df_decisions)):
        minute = float(df_decisions.iloc[i]['minute'])
        second = float(df_decisions.iloc[i]['seconds'])
        index_above = sh[sh['Time'] > float(minute + second/60)].iloc[0].name
        try:
            index_below = sh[sh['Time'] <= float(minute + second/60)].iloc[-1].name
        except:
            index_below = index_above
        index_5min_above = sh.index[sh.index <= index_above +5*60*25].max()
        index_5min_below = sh.index[sh.index >= index_below -5*60*25].min()
        mean_pc_b_home.append(sh.loc[index_below]['PC home previous 5 minutes'])
        mean_wpc_b_home.append(sh.loc[index_below]['Weighted PC home previous 5 minutes'])
        mean_pc_a_home.append(sh.loc[index_5min_above]['PC home previous 5 minutes'])
        mean_wpc_a_home.append(sh.loc[index_5min_above]['Weighted PC home previous 5 minutes'])
        mean_pc_b_away.append(sh.loc[index_below]['PC away previous 5 minutes'])
        mean_wpc_b_away.append(sh.loc[index_below]['Weighted PC away previous 5 minutes'])
        mean_pc_a_away.append(sh.loc[index_5min_above]['PC away previous 5 minutes'])
        mean_wpc_a_away.append(sh.loc[index_5min_above]['Weighted PC away previous 5 minutes'])
        index_min_below.append(index_5min_below)
        index_min_above.append(index_5min_above)



    df_decisions['PC home 5min before'] = mean_pc_b_home
    df_decisions['WPC home 5min before'] = mean_wpc_b_home
    df_decisions['PC home 5min after'] = mean_pc_a_home
    df_decisions['WPC home 5min after'] = mean_wpc_a_home
    df_decisions['PC away 5min before'] = mean_pc_b_away
    df_decisions['WPC away 5min before'] = mean_wpc_b_away
    df_decisions['PC away 5min after'] = mean_pc_a_away
    df_decisions['WPC away 5min after'] = mean_wpc_a_away
    df_decisions['index 5min before'] = index_min_below
    df_decisions['index 5min after'] = index_min_above
    
    minutes_second_half = []

    #check if there are timeframes where no decisions are made using pure playing time. If yes, add these to the dataframe
    for i in range(len(df_decisions)):
        try: 
            if i == 0:
                if df_decisions.iloc[i]['index 5min before'] - sh.iloc[0].name > 20*60*25:
                    dif = df_decisions.iloc[i]['index 5min before'] - sh.iloc[0].name
                    if dif < 30*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/2)]['Time'])
                    elif dif >= 30*60*25 and dif <40*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/3)]['Time'])
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int((dif/3)*2)]['Time'])
                    elif dif >= 40*60*25:
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int(dif/4)]['Time'])
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int((dif/4)*2)]['Time'])
                        minutes_second_half.append(sh.loc[sh.iloc[0].name + int((dif/4)*3)]['Time'])                       

            if df_decisions.iloc[i+1]['index 5min before'] - df_decisions.iloc[i]['index 5min after'] > 20*60*25:
                dif = df_decisions.iloc[i+1]['index 5min before'] - df_decisions.iloc[i]['index 5min after']
                if dif < 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int(dif/2)]['Time'])
                elif dif >= 30*60*25 and dif <40*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int(dif/3)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int((dif/3)*2)]['Time'])
                elif dif >= 40*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int(dif/4)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int((dif/4)*2)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int((dif/4)*3)]['Time'])
        except:
            if sh.iloc[-1].name - df_decisions.iloc[i]['index 5min after'] > 20*60*25:
                dif = sh.iloc[-1].name - df_decisions.iloc[i]['index 5min after']
                if dif < 30*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int(dif/2)]['Time'])
                elif dif >= 30*60*25 and dif <40*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int(dif/3)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int((dif/3)*2)]['Time'])
                elif dif >= 40*60*25:
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int(dif/4)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int((dif/4)*2)]['Time'])
                    minutes_second_half.append(sh.loc[df_decisions.iloc[i]['index 5min after'] + int((dif/4)*3)]['Time'])
                    
    mean_pc_b_home = []
    mean_wpc_b_home = []
    mean_pc_a_home = []
    mean_wpc_a_home = []
    mean_pc_b_away = []
    mean_wpc_b_away = []
    mean_pc_a_away = []
    mean_wpc_a_away = []
    index_min_above = []
    index_min_below = []
    minutes = []
    seconds = []
    half = []
    
    #add all the other data for these new time windows, such as pitch control, and PCV.
    for i in range(len(minutes_second_half)):
        minute = float(int(minutes_second_half[i]))
        second = float((minutes_second_half[i] - int(minutes_second_half[i]))*60)
        index_above = sh[sh['Time'] > float(minute + second/60)].iloc[0].name
        index_below = sh[sh['Time'] <= float(minute + second/60)].iloc[-1].name
        index_5min_below = sh.index[sh.index >= index_below -5*60*25].min()
        index_5min_above = sh.index[sh.index <= index_above +5*60*25].max()
        mean_pc_b_home.append(sh.loc[index_below]['PC home previous 5 minutes'])
        mean_wpc_b_home.append(sh.loc[index_below]['Weighted PC home previous 5 minutes'])
        mean_pc_a_home.append(sh.loc[index_5min_above]['PC home previous 5 minutes'])
        mean_wpc_a_home.append(sh.loc[index_5min_above]['Weighted PC home previous 5 minutes'])
        mean_pc_b_away.append(sh.loc[index_below]['PC away previous 5 minutes'])
        mean_wpc_b_away.append(sh.loc[index_below]['Weighted PC away previous 5 minutes'])
        mean_pc_a_away.append(sh.loc[index_5min_above]['PC away previous 5 minutes'])
        mean_wpc_a_away.append(sh.loc[index_5min_above]['Weighted PC away previous 5 minutes'])
        index_min_below.append(index_5min_below)
        index_min_above.append(index_5min_above)
        minutes.append(minute + 45)
        seconds.append(second)
        half.append(2)
    
    index = []
    type_id = []
    type_name = []
    half = []
    frame_id = []
    team_id = []
    team = []
    formation_change = []
    current_formation_home = []
    current_formation_away = []
    type_of_decision = []
    match_id = []
    half = []
    minute = []
    seconds = []

    for j in range(len(minutes_second_half)):
        index.append(df_decisions.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        match_id.append(df_decisions.iloc[0]['Match'])
        formation_change.append(0)
        frame_id.append(0)
        current_formation_home.append(0)
        current_formation_away.append(0)
        type_of_decision.append('neutral')
        half.append('2')
        minute.append(int(minutes_second_half[j]))
        seconds.append((minutes_second_half[j] - int(minutes_second_half[j]))*60)

    #create dataframe for these new time windows
    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minute,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
            'formation_change' : formation_change,
            'current_formation_home' : current_formation_home,
            'current_formation_away' : current_formation_away,
            'Type of decision' : type_of_decision,
           'Match': match_id,
            'PC home 5min before': mean_pc_b_home,
            'WPC home 5min before': mean_wpc_b_home, 
            'PC home 5min after': mean_pc_a_home,
           'WPC home 5min after': mean_wpc_a_home,
           'PC away 5min before': mean_pc_b_away,
            'WPC away 5min before': mean_wpc_b_away, 
            'PC away 5min after': mean_pc_a_away,
           'WPC away 5min after': mean_wpc_a_away,
            'index 5min before': index_min_below,
            'index 5min after': index_min_above}

    #add these new time windows to the original decision dataframe
    df = pd.DataFrame(data)
    df_new = pd.concat([df_decisions, df])
    
    #import the shots and goals dataframe
    infile = open('shots and goals event dataframes.p','rb')
    df_shot = pickle.load(infile)
    infile.close()
    goals = []
    #create a list with the minute and teams corresponding to the goals
    df_shots = df_shot.loc[df_shot['match_id'] == game]
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['type_id'] == '16':
            goals.append([int(df_shots.iloc[i]['minute']) + int(df_shots.iloc[i]['seconds'])/60, df_shots.iloc[i]['team']])

    time_left = []
    home_goals = []
    away_goals = []
    #for each row in the decision dataframe of a match: add the time left and the home and away score
    for i in range(len(df_new)):
        home = 0
        away = 0
        time_left.append(90 - (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60))
        for j in range(len(goals)):
            if (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Home':
                home = home + 1
            elif (int(df_new.iloc[i]['minute']) + int(df_new.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Away':
                away = away + 1
        home_goals.append(home)
        away_goals.append(away)

    df_new['Time left'] = time_left
    df_new['Home score'] = home_goals
    df_new['Away score'] = away_goals
    #add the match dataframe to the complete dataframe
    final_df = pd.concat([final_df, df_new])
    
#export the decision dataframe for all matches to a pickle file that can be used for regression
pickle.dump(final_df, open('input PC regression decisions on Momentum.p', 'wb'))  


In [ ]:
#five-minute time windows xT
from progressbar import ProgressBar
import pickle
import pandas as pd

#import the decision dataframe for all matches
infile = open('Decision matches dataframe.p','rb')
df_decisions = pickle.load(infile)
infile.close()

#import the xT dataframe for all matches
infile = open('all matches sliding windows xT.p','rb')
df_xT = pickle.load(infile)
infile.close()

#create an array with all matches
matches = df_xT.Match.unique()
pbar = ProgressBar()

#create empty dataframe
final_df = pd.DataFrame()

#iterate over each match in the array
for game in pbar(matches):
    #only take the second half of the match
    match_decisions = df_decisions.loc[df_decisions['Match'] == game]
    match_decisions = match_decisions.loc[match_decisions['half'] == '2']
    
    #check if there are time windows where no decisions are made. If yes, add them to the original decision dataframe
    minutes_second_half = []
    for i in range(len(match_decisions)):
        try:
            if i == 0:
                if (int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60) - 45  > 20:
                    dif = (int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60) - 45
                    if dif < 30:
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - dif/2)
                    elif dif >= 30 and dif <40:
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - dif/3)
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - 2*dif/3)
                    elif dif >= 40:
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - 1*dif/4)
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - 2*dif/4)
                        minutes_second_half.append(int(match_decisions.iloc[i]['minute']) + int(match_decisions.iloc[i]['seconds'])/60 - 3*dif/4)

            if (int(match_decisions.iloc[i+1]['minute'])+int(match_decisions.iloc[i+1]['seconds'])/60) - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60) > 20:
                dif = (int(match_decisions.iloc[i+1]['minute'])+int(match_decisions.iloc[i+1]['seconds'])/60) - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60)
                if dif < 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 30 and dif <40:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 40:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/4)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 3*dif/4)

        except:
            if 91 - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60) > 20:
                dif = 91 - (int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60)
                if dif < 30:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 30 and dif <40:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 40:
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 1*dif/4)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_second_half.append(int(match_decisions.iloc[i]['minute'])+int(match_decisions.iloc[i]['seconds'])/60 + 3*dif/4)
        

    match_decisions = match_decisions[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'formation_change', 'current_formation_home', 'current_formation_away', 'Type of decision', 'Match']]

    index = []
    type_id = []
    type_name = []
    half = []
    frame_id = []
    team_id = []
    team = []
    formation_change = []
    current_formation_home = []
    current_formation_away = []
    type_of_decision = []
    match_id = []
    half = []
    minute = []
    seconds = []

    #add other data for these new time windows
    for j in range(len(minutes_second_half)):
        index.append(match_decisions.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        match_id.append(match_decisions.iloc[0]['Match'])
        formation_change.append(0)
        frame_id.append(0)
        current_formation_home.append(0)
        current_formation_away.append(0)
        type_of_decision.append('neutral')
        half.append('2')
        minute.append(int(minutes_second_half[j]))
        seconds.append((minutes_second_half[j] - int(minutes_second_half[j]))*60)

    #create a dataframe for the new time windows
    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minute,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
            'formation_change' : formation_change,
            'current_formation_home' : current_formation_home,
            'current_formation_away' : current_formation_away,
            'Type of decision' : type_of_decision,
           'Match': match_id}


    df_new = pd.DataFrame(data)
    #add this dataframe to the dataframe of the match
    df = pd.concat([match_decisions, df_new])
    
    #open the shots and goals dataframe
    infile = open('shots and goals event dataframes.p','rb')
    df_shot = pickle.load(infile)
    infile.close()
    goals = []
    #create a list with the mnute and team corresponding to the goals scored
    df_shots = df_shot.loc[df_shot['match_id'] == game]
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['type_id'] == '16':
            goals.append([int(df_shots.iloc[i]['minute']) + int(df_shots.iloc[i]['seconds'])/60, df_shots.iloc[i]['team']])

    time_left = []
    home_goals = []
    away_goals = []
    
    #for each row in the decision dataframe: add the time left and the home and away score
    for i in range(len(df)):
        home = 0
        away = 0
        time_left.append(90 - (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60))
        for j in range(len(goals)):
            if (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Home':
                home = home + 1
            elif (int(df.iloc[i]['minute']) + int(df.iloc[i]['seconds'])/60) >= goals[j][0] and goals[j][1] == 'Away':
                away = away + 1
        home_goals.append(home)
        away_goals.append(away)
    
    df['Time left'] = time_left
    df['Home score'] = home_goals
    df['Away score'] = away_goals
    
    #create a final dataframe for all the matches but still without the xT calculations
    final_df = pd.concat([final_df, df])


In [ ]:
pbar = ProgressBar()
#create empty dataframe
df_final = pd.DataFrame()
#iterate over all matches
for game in pbar(matches):
    #take again only the second half decisions
    df_decisions = final_df.loc[final_df['Match'] == game]
    df_decisions['minute'] = pd.to_numeric(df_decisions['minute'])
    df_decisions = df_decisions.loc[df_decisions['minute'] >= 50]
    df_match_xT = df_xT.loc[df_xT['Match'] == game]
    df_xT_sh = df_match_xT.loc[df_match_xT['Half'] == 2]
    xt_home_5minb = []
    xt_away_5minb = []
    xt_home_5mina = []
    xt_away_5mina = []
    #for each row in the dataframe: add the xT values for the time windows
    for i in range(len(df_decisions)):
        time = int(df_decisions.iloc[i]['minute']) + int(df_decisions.iloc[i]['seconds'])/60
        xt_home_5minb.append(df_xT_sh.loc[df_xT_sh['Minute'] <= time].iloc[-1]['xT_prev_5min_home'])
        xt_away_5minb.append(df_xT_sh.loc[df_xT_sh['Minute'] <= time].iloc[-1]['xT_prev_5min_away'])
        xt_home_5mina.append(df_xT_sh.loc[df_xT_sh['Minute'] <= time + 3].iloc[-1]['xT_prev_5min_home'])
        xt_away_5mina.append(df_xT_sh.loc[df_xT_sh['Minute'] <= time + 3].iloc[-1]['xT_prev_5min_away'])
    
    #add the xT values to the dataframe
    df_decisions['xT_prev_5min_home'] = xt_home_5minb
    df_decisions['xT_next_5min_home'] = xt_home_5mina
    df_decisions['xT_prev_5min_away'] = xt_away_5minb
    df_decisions['xT_next_5min_away'] = xt_away_5mina
    
    #create a complete dataframe for each match including xT values
    df_final = pd.concat([df_decisions, df_final])
        
#export the complete dataframe for all the matches to a pickle file that can be used for regression
pickle.dump(df_final, open('input xT regression decisions on Momentum.p', 'wb')) 